In [1]:
import os
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


pd_data_dir = "/home/djl34/lab_pd/data"
aso_data_dir = "/home/djl34/lab_pd/aso/data"
KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch3/users/d/djl34"

## get a list of genes that are AD and lof

In [2]:
filename = aso_data_dir + "/Clingen-Gene-Disease-Summary-2023-06-24.csv"
df = pd.read_csv(filename)

filename = aso_data_dir + "/Clingen-Dosage-Sensitivity-2023-06-25.csv"
df_ds = pd.read_csv(filename)
df_ds_haploinsufficient = df_ds[df_ds["HAPLOINSUFFICIENCY"].isin(['Sufficient Evidence for Haploinsufficiency', 'Emerging Evidence for Haploinsufficiency'])]

In [3]:
df_ds["HAPLOINSUFFICIENCY"].unique()

array(['Gene Associated with Autosomal Recessive Phenotype',
       'Sufficient Evidence for Haploinsufficiency',
       'No Evidence for Haploinsufficiency',
       'Little Evidence for Haploinsufficiency',
       'Emerging Evidence for Haploinsufficiency',
       'Dosage Sensitivity Unlikely'], dtype=object)

In [4]:
df["CLASSIFICATION"].unique()

array(['Disputed', 'Definitive', 'Moderate', 'Limited',
       'No Known Disease Relationship', 'Strong', 'Refuted'], dtype=object)

In [5]:
df_AD = df[df["MOI"] == "AD"]

df_AD_strong = df_AD[df_AD["CLASSIFICATION"].isin(["Strong", "Definitive"])]

df_AD_strong_lof = df_AD_strong.merge(df_ds_haploinsufficient, on = "GENE SYMBOL", how = "inner")

In [6]:
gene_list = list(df_AD_strong_lof["GENE SYMBOL"].unique())

In [7]:
len(gene_list)

191

In [8]:
gene_list

['ACVRL1',
 'ADNP',
 'AHDC1',
 'ANK2',
 'ANKRD11',
 'ANKRD17',
 'APC',
 'APOB',
 'ARID1A',
 'ARID1B',
 'ARID2',
 'ASH1L',
 'ASXL1',
 'ASXL3',
 'ATM',
 'AUTS2',
 'BAG3',
 'BAP1',
 'BARD1',
 'BCL11A',
 'BMPR1A',
 'BMPR2',
 'BPTF',
 'BRCA1',
 'BRCA2',
 'BRIP1',
 'CDC73',
 'CDH1',
 'CDK13',
 'CDKN1B',
 'CDKN2A',
 'CHD2',
 'CHD7',
 'CHD8',
 'CHEK2',
 'COL1A1',
 'COL2A1',
 'COL3A1',
 'COL5A1',
 'CREBBP',
 'CSNK2A1',
 'CTCF',
 'CTNNB1',
 'CUL3',
 'CYLD',
 'DDX41',
 'DICER1',
 'DSC2',
 'DSG2',
 'DSP',
 'DYRK1A',
 'EFTUD2',
 'EHMT1',
 'ENG',
 'EP300',
 'ERF',
 'ETV6',
 'EXT1',
 'EXT2',
 'EYA1',
 'EYA4',
 'FBN1',
 'FGFR1',
 'FH',
 'FLCN',
 'FLNC',
 'FOXC1',
 'FOXG1',
 'FOXP1',
 'GATA2',
 'GATA3',
 'GATA4',
 'GATAD2B',
 'GRIN2A',
 'GRIN2B',
 'HNF1B',
 'HNRNPK',
 'KANSL1',
 'KAT6A',
 'KAT6B',
 'KCNH2',
 'KCNQ1',
 'KCNQ2',
 'KIF11',
 'KMT2A',
 'KMT2B',
 'KMT2C',
 'KMT2D',
 'KMT2E',
 'KMT5B',
 'LMNA',
 'LMX1B',
 'MAX',
 'MBD5',
 'MED13L',
 'MEF2C',
 'MEIS2',
 'MEN1',
 'MITF',
 'MLH1',
 'MSH2',
 'MSH

In [13]:
filename = pd_data_dir + "/biomart/ENSG_Genename_syn.tsv"
df_ensg = pd.read_csv(filename, sep = "\t")
df_ensg = df_ensg.rename({"Gene name": "GENE SYMBOL", "Gene stable ID" : "Gene"}, axis = 1)
df_ensg = df_ensg[["GENE SYMBOL", "Gene", "Chromosome/scaffold name"]].drop_duplicates()
df_ensg = df_ensg[df_ensg["Chromosome/scaffold name"].str.contains("CHR") == False]
df_ensg = df_ensg.rename({"Gene name": "GENE SYMBOL", "Gene stable ID" : "Gene"}, axis = 1)

df_AD_strong_lof_ensg = df_AD_strong_lof.merge(df_ensg, on = "GENE SYMBOL", how = "left")

ensg_gene_list = list(df_AD_strong_lof_ensg["Gene"].unique())

## combine with variant data

In [9]:
client = Client()

In [11]:
chrom = str(22)

variants = os.path.join(aso_data_dir, "whole_gene/spliceai/"+ chrom +"/_metadata")
variants = dd.read_parquet("/".join(variants.split("/")[:-1]) + "/")

In [12]:
variants.head()

,CHROM,Pos,Allele_ref,Allele,Consequence,Gene,Feature,MaxEntScan_alt,MaxEntScan_diff,MaxEntScan_ref,LaBranchoR_start,LaBranchoR_score,spliceai_gene,DS,DS_AG,DS_AL,DS_DG,DS_DL
0,22,10736171,T,A,non_coding_transcript_exon_variant,ENSG00000277248,ENST00000615943,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
1,22,10736171,T,C,non_coding_transcript_exon_variant,ENSG00000277248,ENST00000615943,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
2,22,10736171,T,G,non_coding_transcript_exon_variant,ENSG00000277248,ENST00000615943,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
3,22,10736172,T,A,non_coding_transcript_exon_variant,ENSG00000277248,ENST00000615943,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
4,22,10736172,T,C,non_coding_transcript_exon_variant,ENSG00000277248,ENST00000615943,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0


In [18]:
variants_ad = variants[variants["Gene"].isin(ensg_gene_list)]

In [20]:
## we get a list of variants, in a certain chromosome that are ad strong genes

In [19]:
variants_ad = variants_ad.compute()

In [22]:
len(list(variants_ad["Gene"].unique()))

8

In [21]:
variants_ad

,CHROM,Pos,Allele_ref,Allele,Consequence,Gene,Feature,MaxEntScan_alt,MaxEntScan_diff,MaxEntScan_ref,LaBranchoR_start,LaBranchoR_score,spliceai_gene,DS,DS_AG,DS_AL,DS_DG,DS_DL
3513549,22,28687743,C,A,3_prime_UTR_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
3513550,22,28687743,C,G,3_prime_UTR_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
3513551,22,28687743,C,T,3_prime_UTR_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
3513552,22,28687744,C,A,3_prime_UTR_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
3513553,22,28687744,C,G,3_prime_UTR_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369931,22,23829501,C,G,intron_variant,ENSG00000099956,ENST00000644036,.,.,.,NaN,NaN,SMARCB1,0.0,0.0,0.0,0.0,0.0
369932,22,23829501,C,T,intron_variant,ENSG00000099956,ENST00000644036,.,.,.,NaN,NaN,SMARCB1,0.0,0.0,0.0,0.0,0.0
369933,22,23829502,C,A,intron_variant,ENSG00000099956,ENST00000644036,.,.,.,NaN,NaN,SMARCB1,0.0,0.0,0.0,0.0,0.0
369934,22,23829502,C,G,intron_variant,ENSG00000099956,ENST00000644036,.,.,.,NaN,NaN,SMARCB1,0.0,0.0,0.0,0.0,0.0


In [39]:
len(variants_ad[variants_ad["Consequence"] == "splice_donor_variant"])

684

In [58]:
variants_ad[variants_ad["Consequence"] == "splice_donor_variant"].value_counts("DS_DL")

DS_DL
1.00    166
0.99     99
0.98     52
0.00     48
1.98     48
2.00     48
0.97     31
0.96     19
0.90     18
1.96     15
0.93     10
0.94     10
1.92      9
0.86      9
1.76      7
0.76      7
1.70      6
0.41      6
0.50      6
0.66      6
0.95      5
0.85      5
1.94      4
0.89      4
0.16      3
0.56      3
0.55      3
1.90      3
1.86      2
0.92      2
0.82      2
0.74      2
0.84      2
0.28      1
0.15      1
0.09      1
0.29      1
0.30      1
0.37      1
0.47      1
0.52      1
0.54      1
0.61      1
1.44      1
0.13      1
0.65      1
0.67      1
0.73      1
0.78      1
0.79      1
0.80      1
0.91      1
0.06      1
0.83      1
0.88      1
0.87      1
0.18      1
dtype: int64

In [40]:
len(variants_ad[variants_ad["Consequence"] == "splice_acceptor_variant"])

696

## which of these variants are gain of misplicing?

spliceAI donor/ acceptor gain score at a non-canonical site ≥ 0.1 AND (ii) MaxEntScan donor/acceptor score with the ALT allele at the site ≥ 2.

In [30]:
misplicing_gain = variants_ad[(variants_ad["DS_AG"] > 0.1) | (variants_ad["DS_DG"] > 0.1)]

In [47]:
misplicing_gain["MaxEntScan_alt"] = misplicing_gain["MaxEntScan_alt"].replace(".", None)
misplicing_gain["MaxEntScan_alt"] = misplicing_gain["MaxEntScan_alt"].astype(float)

misplicing_gain["MaxEntScan_ref"] = misplicing_gain["MaxEntScan_ref"].replace(".", None)
misplicing_gain["MaxEntScan_ref"] = misplicing_gain["MaxEntScan_ref"].astype(float)

/tmp/ipykernel_27022/706118262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misplicing_gain["MaxEntScan_alt"] = misplicing_gain["MaxEntScan_alt"].replace(".", None)
/tmp/ipykernel_27022/706118262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misplicing_gain["MaxEntScan_alt"] = misplicing_gain["MaxEntScan_alt"].astype(float)
/tmp/ipykernel_27022/706118262.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [43]:
misplicing_gain = misplicing_gain[misplicing_gain["MaxEntScan_alt"] > 2]

need to filter out severe splice destroying variants: SpliceAI donor/acceptor loss score ≥ 0.1 at a canonical splice site) AND (MaxEntScan donor/ acceptor score with the ALT allele < MaxEntScan donor/acceptor score with the REF allele) AND [(MaxEntScan donor/acceptor score with the ALT allele < 2) OR (MaxEntScan donor/acceptor score with the ALT allele < 0.3 × MaxEntScan donor/acceptor score with the REF allele)].

In [51]:
## remove non-severe

misplicing_gain = misplicing_gain[(((misplicing_gain["DS_AL"] > 0.1) |(misplicing_gain["DS_DL"] > 0.1)) & 
                (misplicing_gain["MaxEntScan_alt"] < misplicing_gain["MaxEntScan_ref"]) & 
                ((misplicing_gain["MaxEntScan_alt"] < 2) | 
                 (misplicing_gain["MaxEntScan_alt"] < 0.3 * misplicing_gain["MaxEntScan_ref"]))) == False]

In [54]:
misplicing_gain["Consequence"].unique()

array(['splice_polypyrimidine_tract_variant&intron_variant',
       'intron_variant',
       'splice_region_variant&splice_polypyrimidine_tract_variant&intron_variant',
       'splice_acceptor_variant',
       'splice_donor_region_variant&intron_variant',
       'splice_donor_5th_base_variant&intron_variant',
       'splice_donor_variant'], dtype=object)

In [56]:
misplicing_gain.value_counts("Consequence")

Consequence
splice_polypyrimidine_tract_variant&intron_variant                          242
splice_region_variant&splice_polypyrimidine_tract_variant&intron_variant    211
splice_donor_region_variant&intron_variant                                  163
intron_variant                                                               82
splice_donor_5th_base_variant&intron_variant                                 52
splice_acceptor_variant                                                      32
splice_donor_variant                                                          5
dtype: int64

In [55]:
len(misplicing_gain)

787

## which of these are exon skipping or intron retention?

(2) Exon skipping or intron retention (skipping or retention): SpliceAI donor/acceptor loss score at any canonical site ≥ 0.1 without an accompanying gain of mis-splicing by SpliceAI (donor/acceptor gain score < 0.1 at any non-canonical splice site).

In [41]:
exon_skip_intron_ret = variants_ad[(variants_ad["DS_AL"] > 0.1) | (variants_ad["DS_DL"] > 0.1)]

In [42]:
exon_skip_intron_ret

,CHROM,Pos,Allele_ref,Allele,Consequence,Gene,Feature,MaxEntScan_alt,MaxEntScan_diff,MaxEntScan_ref,LaBranchoR_start,LaBranchoR_score,spliceai_gene,DS,DS_AG,DS_AL,DS_DG,DS_DL
3518533,22,28689404,G,C,intron_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,CHEK2CHEK2,0.28,0.0,0.0,0.28,0.14
3518616,22,28689432,A,C,intron_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,CHEK2CHEK2,0.16,0.0,0.0,0.06,0.16
3518617,22,28689432,A,G,intron_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,CHEK2CHEK2,0.16,0.0,0.0,0.14,0.16
3518618,22,28689432,A,T,intron_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,CHEK2CHEK2,0.16,0.0,0.0,0.12,0.16
3518619,22,28689433,C,A,intron_variant,ENSG00000183765,ENST00000404276,.,.,.,NaN,NaN,CHEK2CHEK2,0.16,0.0,0.0,0.16,0.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357676,22,23825416,G,C,splice_donor_variant,ENSG00000099956,ENST00000644036,1.513,8.273,9.786,NaN,NaN,SMARCB1,1.00,0.0,0.0,0.01,1.00
357677,22,23825416,G,T,splice_donor_variant,ENSG00000099956,ENST00000644036,1.282,8.504,9.786,NaN,NaN,SMARCB1,1.00,0.0,0.0,0.32,1.00
357678,22,23825417,T,A,splice_donor_variant,ENSG00000099956,ENST00000644036,1.602,8.183,9.786,NaN,NaN,SMARCB1,1.00,0.0,0.0,0.01,1.00
357679,22,23825417,T,C,splice_donor_variant,ENSG00000099956,ENST00000644036,2.032,7.754,9.786,NaN,NaN,SMARCB1,0.56,0.0,0.0,0.01,0.56
